# Operators

## Identifiers

* Alphanumeric, underscore and operator characters can be
used in identifiers.
* Identifiers can also contain mathematical symbols and
unicode characters.
* If identifier names are same as reserved words in scala,
then such identifiers are enclosed within backticks.

In [1]:
val `val` = 1
println(`val`)

1


`val`: Int = 1


## Infix operators

In [2]:
//This is what actually happens
println(1.+(2))

// This notation is called infix notation
println(1  + 2)
println(1 to 5)

3
3
Range 1 to 5


In [4]:
// Unary operators are prefix operators
// unary operators are + - !, ~. Other characters are not allowed
// to define unary operator, 
// create a method called unary_{!|-|!|~}

class Temperature(val celcius: Int)  { //extends AnyVal {
    def unary_- : Temperature = new Temperature(-this.celcius)    
    def apply(): Int = this.celcius
}

val temp = new Temperature(100)
println(temp())
val negTemp = -temp
println(negTemp())

100
-100


defined class Temperature
temp: Temperature = ammonite.$sess.cmd3$Helper$Temperature@4972d3b7
negTemp: Temperature = ammonite.$sess.cmd3$Helper$Temperature@1e63f505

**NOTE**: Avoid using operators with postfix notation. `obj op` is the
postfix notation which is equivalent to `obj.op`


## Compound assignment

* Operators of the form `operator=` are compound assignment operators.
* Compound assignment is commonly used with arithmetic operators.

## Operator precedence

* `Prec(postfix) < Prec(infix) < Prec(Prefix)`
* `a infix b postfix` is same as `(a infix b) postfix)`
* For operator precedence table refer the book.

## Associativity

* Operators that end with `:` and assignment operators are right
associative, all others are left associative.

* Right associative operators are methods of the right operand in the
infix notation. Ex `a right_assoc_op b` is `b.right_assoc_op(a)`

## `apply` and `update` methods

* class or instance of a class with `apply` methods make them look
like functions, meaning they can be invoked like a function call.
* `update` method helps assignment like `obj(x) = y`

In [8]:
class Item(val name: String,
           val price: Double,
           var quantity: Int = 0) {
    override def toString = s"Item($name, $price, $quantity)"
}

object Bag {
    // This apply method can help us instantiate item
    // without using new keyword
    def apply(items: Iterable[Item]): Bag = {
        new Bag(items.toList)
    }
}

class Bag(val items: List[Item]) {
    private val nameToItemMap = items.map(i => (i.name, i)).toMap
    // this will clone this item unit times.
    def apply(name: String): Option[Item] = nameToItemMap.get(name)
    
    def update(name: String, quantity: Int): Unit = {
        nameToItemMap.get(name).map(i => i.quantity = quantity)
    }
}

// Notice we call class like function without new
val items = List[Item](
    new Item("Snickers", 2.0, 1),
    new Item("Kitkat", 1.0, 2)
)
val chocoBag = Bag(items) // Class instantiated like function
println(chocoBag("Kitkat"))
chocoBag("Kitkat") = 5

for (choc <- chocoBag.items)
    println(choc)

Some(Item(Kitkat, 1.0, 2))
Item(Snickers, 2.0, 1)
Item(Kitkat, 1.0, 5)


defined class Item
defined object Bag
defined class Bag
items: List[Item] = List(Item(Snickers, 2.0, 1), Item(Kitkat, 1.0, 5))
chocoBag: Bag = ammonite.$sess.cmd7$Helper$Bag@41b6439e

## Extractors

* Opposite of `apply` method
* `unapply` method on the instance or class present on the left side of the assignment is
called with the object on the right side of the assignment.
* Return value of the unapply method is assigned to the parameters
mentioned on the left side.

In [10]:
object Item {
    def apply(name: String, price: Double, quantity: Int) = {
        new Item(name, price, quantity)
    }
    
    def unapply(item: Item): Option[(String, Double, Int)]  = {
        Some(item.name, item.price, item.quantity)
    }
}

class Item(val name: String,
           val price: Double,
           var quantity: Int = 0) {
    override def toString = s"Item($name, $price, $quantity)"
}

val item = Item("5star", 5.0, 1)

//extract fields from Item using extractor pattern
val Item(name, price, quantity) = item
println(name)

// we could also use this in pattern matching
// case classes come with apply and unapply methods supplied by default
item match {
    case Item(name, price, _) => println(s"$name, $price")
    case _ => ()
}

5star
5star, 5.0


defined object Item
defined class Item
item: Item = Item(5star, 5.0, 1)
name: String = "5star"
price: Double = 5.0
quantity: Int = 1

In [11]:
//  We can write any custom `object` to extract information 
// from any instance of a class.
// Custom extractor object for Item objects
object ItemQuantity {
    def unapply(input: String): Option[(String, Int)] = {
        val parts = input.split(" ")
        Some((parts(1), parts(0).toInt))
    }
}

val ItemQuantity(name, nos) = "100 apples"
println(s"$name, $nos")

apples, 100


defined object ItemQuantity
name: String = "apples"
nos: Int = 100

In [14]:
// This mechanism is used in extracting regex patterns which contain
// named groups - Refer chapter-9
class Item(val name: String, val price: Double) {
    def unapply(item: Item): Option[(String, Double)] = {
        Some(this.name, this.price)
    }
}

object PriceChecker {
    def unapply(item: Item): Boolean = {
        item.price > 5.0
    }
}

val item = new Item("a", 10.0)
// we can also define unapply method on instances
val item(name, price) = item
println(name, price)

item match {
    // extractor with no arguments
    case PriceChecker() => println("Item is pricey")
    case _ => println("Item is not pricey")
}

(a,4.0)
Item is not pricey


defined class Item
defined object PriceChecker
item: Item = ammonite.$sess.cmd13$Helper$Item@41351a0b
name: String = "a"
price: Double = 4.0

**NOTE**: When `unapply` method returns a Boolean, we can have 
the extractors with no arguments

* If we extract a collection of values from an input object, then we should
define `unapplySeq` to do so.

In [19]:
class Item(val name: String, val quantity: Int) 

object ItemExtractor {
    def unapplySeq(input: String): Option[Seq[Item]] = {
        val items = input.split(",")
        if (items.length == 0)
            None
        else
            Some(items.map(i => {
              val parts = i.trim().split(" ")
              new Item(parts(1), parts(0).toInt)
            }).toSeq)
    }
}
                 
val ItemExtractor(item1, item2) = "10 apples, 10 mangoes"
println(s"${item1.name}, ${item2.name}")

apples, mangoes


defined class Item
defined object ItemExtractor
item1: Item = ammonite.$sess.cmd18$Helper$Item@2aacaad6
item2: Item = ammonite.$sess.cmd18$Helper$Item@ef1b641

## Dynamic Invocation

> If a type extends the trait `scala.Dynamic`, then method
calls, getters, and setters are rewritten as calls to special methods that can inspect
the name of the original call and the parameters, and then take arbitrary actions

* This feature has to be explicitly enabled by importing
`import scala.language.dynamics`

Dynamic works in the following way:
* `obj.someParameterLessMethodOrField` - calls 
`obj.selectDynamic("someParameterLessMethodOrField)"`

* `obj.someMember(arg1, arg2)` - when all args are positional only
, then calls `obj.applyDynamic("someMember")(arg1, arg2)`

* `obj.someMember(p1=arg1, p2=arg2)` - calls 
`obj.applyDynamicNamed(someMember)((p1, arg1), (p2. arg2))`

* `obj.someMember = expr` - becomes rewritten to 
`obj.updateDynamic("someMember")(expr)`

In [21]:
import scala.language.dynamics
import scala.collection.mutable.{Map => MutableMap}

// This of this like implementing __getattr__ and __setattr__ in python
// to set and access custom attributes on the object

// To make this implementation generic, we might have to extensively use
// generic types.
class DynamicExample extends Dynamic {
    private val nameActionMap = MutableMap[String, (String => Unit)]()
    
    def selectDynamic(member: String) = {
        println("Inside selectDynamic")
        nameActionMap.get(member)
    }
    
    def applyDynamic(member: String)(arg: String) = {
        println("Inside applyDynamic")
        nameActionMap.get(member).foreach(l => l(arg))
    }
    
    def updateDynamic(member: String)(lambda: (String => Unit)) = {
        println("Inside updateDynamic")
        nameActionMap += (member -> lambda)
    }
}

val dynEx = new DynamicExample

// calls updateDynamic
dynEx.printMessage = (msg => println(msg))

// calls applyDynamic
dynEx.printMessage("Hello world")

// calls selectDynamic
val action = dynEx.printMessage
action.foreach(l => l("foo bar"))

Inside updateDynamic
Inside applyDynamic
Hello world
Inside selectDynamic
foo bar


import scala.language.dynamics

import scala.collection.mutable.{Map => MutableMap}


defined class DynamicExample
dynEx: DynamicExample = ammonite.$sess.cmd20$Helper$DynamicExample@c66c390
res20_4: collection.mutable.Map[String, String => Unit] = HashMap(
  "printMessage" -> ammonite.$sess.cmd20$Helper$$Lambda$2830/737360600@3a32f6d9
)
action: Option[String => Unit] = Some(
  ammonite.$sess.cmd20$Helper$$Lambda$2830/737360600@3a32f6d9
)